In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
def get_data(url):
    data = {'main_url': [], 'title': [], 'rating': [], 'price': [], 'main_image_url': [], 'other_image_urls': []}
    data['main_url'].append(url)

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('Failed to load page')
    soup = BeautifulSoup(response.text, 'lxml')

    # get title
    title_element = soup.select_one('#productTitle')
    title = title_element.text.strip()
    data['title'].append(title)

    # get rating
    rating_element = soup.select_one('#acrPopover')
    rating_text = rating_element.attrs['title']
    rating = float(rating_text.split()[0])
    data['rating'].append(rating)

    # get price
    price_element = soup.select_one('span.a-offscreen')
    price_element.text
    price = float(price_element.text[1:])
    data['price'].append(price)

    # get main image
    image_element = soup.select_one('#landingImage')
    main_image = image_element.attrs.get('src')
    data['main_image_url'].append(main_image)

    # get the rest of the images
    text = response.text.replace('\t', '').replace(' ', '')
    start_text = """P.when('A').register("ImageBlockATF",function(A){\nvardata={"""
    end_text = """};\nA.trigger('P.AboveTheFold');//triggerATFevent.\nreturndata;\n});\n</script>"""
    assert(start_text in text)
    assert(end_text in text)
    text_body = text[text.find(start_text) + len(start_text):]
    text_body = text_body[:text_body.find(end_text)]

    lines = [line.strip() for line in text_body.split('\n') if line.strip()]
    lines = [line for line in lines if 'colorImages' in line]
    assert(len(lines) == 1)
    text = lines[0]
    # extract all the image urls
    image_urls = []
    for line in text.split('['):
        if 'https' in line:
            image_url = line.split('"')[1]
            if 'https' in image_url:
                image_urls.append(image_url)
                assert('https://m.media-amazon.com/images/I/' in image_url)

    other_image_urls = []
    bases = set([get_base(url) for url in image_urls])
    for base in bases:
        max_size = 0
        max_size_url = None
        for url in [x for x in image_urls if get_base(x) == base]:
            size = int(get_size(url)[1])
            if size > max_size:
                max_size = size
                max_size_url = url
        other_image_urls.append(max_size_url)
    data['other_image_urls'].append(other_image_urls)
    return pd.DataFrame(data)

In [33]:
### Not Ties
# url = 'https://www.amazon.com/Bose-QuietComfort-45-Bluetooth-Canceling-Headphones/dp/B098FKXT8L'
# url = 'https://www.amazon.com/Purina-Pro-Plan-Shredded-Chicken/dp/B07116Y581?pd_rd_w=euQth&content-id=amzn1.sym.1c5f8f79-ee2f-4fb3-a1b5-3d8384cdfaf3&pf_rd_p=1c5f8f79-ee2f-4fb3-a1b5-3d8384cdfaf3&pf_rd_r=1NZ9GR7PWN0FHF2RQ42S&pd_rd_wg=PJ1jM&pd_rd_r=55ba92d0-6945-4ec5-b3fe-2b1908e88225&pd_rd_i=B07116Y581&ref_=pd_bap_d_grid_rp_0_1_ec_scp_pd_hp_d_atf_rp_1_i&th=1'
# url = 'https://www.amazon.com/dp/B0BBXY8QT6?psc=1&ref=ppx_yo2ov_dt_b_product_details'

# Ties
# url = 'https://www.amazon.com/Adulove-Necktie-Classic-Woven-Jacquard/dp/B07BVV8ZNQ'
url = 'https://www.amazon.com/DAZI-Necktie-Weddings-Groomsmen-Missions/dp/B07K4RXQTH'
df = get_data(url)

In [34]:
name = url.split('/')[-1]
df_path = f'dataset_v0/dataframes/{name}.csv'
df.to_csv(df_path, index=False)

In [35]:
df = pd.read_csv(df_path)
x = df.iloc[0]['other_image_urls']

In [36]:
eval(x)[0]

'https://m.media-amazon.com/images/I/61vqpouz9fL._AC_SY879_.jpg'